# Import the libraries to use

In [10]:
#Import system manupulation libraries
import os
import shutil

#Import data manipulation libraries
import numpy as np
import pandas as pd

# Load data and Division of data

In [2]:
images_df = pd.read_csv('../data/raw/train.csv')
images_df.head()

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1


## Separation of datasets

In [3]:
def separate_dataframes(dataframe, separation_percentage, shuffle=False):
    manual_seed = 42
    separation_marker = int(len(dataframe) * separation_percentage)
    
    if shuffle:
        dataframe = dataframe.sample(frac=1, random_state = manual_seed)
    
    first_dataframe = dataframe.iloc[0:separation_marker]
    second_dataframe = dataframe.iloc[separation_marker:len(dataframe)]
    
    return first_dataframe, second_dataframe

### Separation of validation and experimental images

In [4]:
validation_images_df, experiment_images_df = separate_dataframes(images_df, 0.1, shuffle=True)
validation_images_df.shape, experiment_images_df.shape

((1750, 2), (15750, 2))

In [5]:
experiment_images_df.head()

,id,has_cactus
8270,76bf341d7df751ef80b909162d14c607.jpg,0
6827,624ec1063478e4139276c34bef65f1c1.jpg,1
5714,526e960fe6f98ced0232209623e0a2e6.jpg,1
10268,93e1cdb4f20c07ae49801e4e40fdfced.jpg,1
4040,38a695da227092bceca0003c32e0e49e.jpg,1


In [6]:
validation_images_df.head()

,id,has_cactus
1411,13f46a0f794649dad695a7e411bc8cd6.jpg,0
7555,6cab3d7c31c2ce21c3b404892657ecb7.jpg,0
4930,46a1b5d08220741fc8a4a53e1069df6a.jpg,0
10054,90dcaf29c59df190744450761c5eae9b.jpg,1
2714,26142c37accad5e00f8b55d7cb8f9fb7.jpg,1


### Separation of training and test images

In [7]:
test_df, train_df = separate_dataframes(experiment_images_df, 0.2)
test_df.shape, train_df.shape

((3150, 2), (12600, 2))

In [8]:
test_df.head()

,id,has_cactus
8270,76bf341d7df751ef80b909162d14c607.jpg,0
6827,624ec1063478e4139276c34bef65f1c1.jpg,1
5714,526e960fe6f98ced0232209623e0a2e6.jpg,1
10268,93e1cdb4f20c07ae49801e4e40fdfced.jpg,1
4040,38a695da227092bceca0003c32e0e49e.jpg,1


In [9]:
train_df.head()

,id,has_cactus
1801,1937d8278a8c1c58b30898c8d1ea6685.jpg,1
4756,43f3e47323b9c6df111abf570872980f.jpg,1
16222,ecc886298e55127281cca7de6a12a6b9.jpg,1
15119,dc6bf70a7d30760d559477973d525915.jpg,1
530,07ba973d4f1a3f7ace8df35caaec3085.jpg,0


# Export images by category

In [42]:
def export_images(dataset, original_path, new_path, flag=None):
    if flag is not None:
        category1_path = os.path.join(new_path, 'has_cactus')
        os.makedirs(category1_path)
        print(f'Dif: {category1_path} created.')
        category2_path = os.path.join(new_path, 'no_cactus')
        os.makedirs(category2_path)
        print(f'Dif: {category2_path} created.')
    for row in dataset.itertuples(index=False):
        if flag is None:
            shutil.copyfile(os.path.join(original_path, row[0]), os.path.join(new_path, row[0]))
        elif row[1] == flag:
            shutil.copyfile(os.path.join(original_path, row[0]), os.path.join(category1_path, row[0]))
        else:
            shutil.copyfile(os.path.join(original_path, row[0]), os.path.join(category2_path, row[0]))

In [37]:
original_path = '/home/jascrer/TFM/TFM_DL_CNN_Codigo_SeguraCampos/tfm_dl-cnn_codigo_seguracampos/data/raw/train/'
new_val_path = '/home/jascrer/TFM/TFM_DL_CNN_Codigo_SeguraCampos/tfm_dl-cnn_codigo_seguracampos/data/interim/validation'
new_test_path = '/home/jascrer/TFM/TFM_DL_CNN_Codigo_SeguraCampos/tfm_dl-cnn_codigo_seguracampos/data/interim/experiment/test/'
new_train_path = '/home/jascrer/TFM/TFM_DL_CNN_Codigo_SeguraCampos/tfm_dl-cnn_codigo_seguracampos/data/interim/experiment/train/'

## Validation Images Export

In [39]:
import time
start_time = time.time()

os.makedirs(new_val_path)
export_images(validation_images_df, original_path, new_val_path)

finish_time = time.time()
print(f'Validation export lasted: {finish_time-start_time} seconds.')

Validation export lasted: 11.698983907699585 seconds.


## Experiment Images Export

### Training Images Export

In [44]:
import time
start_time = time.time()

os.makedirs(new_train_path)
export_images(train_df, original_path, new_train_path, flag=1)

finish_time = time.time()
print(f'Validation export lasted: {finish_time-start_time} seconds.')

Dif: /home/jascrer/TFM/TFM_DL_CNN_Codigo_SeguraCampos/tfm_dl-cnn_codigo_seguracampos/data/interim/experiment/train/has_cactus created.
Dif: /home/jascrer/TFM/TFM_DL_CNN_Codigo_SeguraCampos/tfm_dl-cnn_codigo_seguracampos/data/interim/experiment/train/no_cactus created.
Validation export lasted: 0.8234012126922607 seconds.


### Test Images Export

In [45]:
import time
start_time = time.time()

os.makedirs(new_test_path)
export_images(test_df, original_path, new_test_path, flag=1)

finish_time = time.time()
print(f'Validation export lasted: {finish_time-start_time} seconds.')

Dif: /home/jascrer/TFM/TFM_DL_CNN_Codigo_SeguraCampos/tfm_dl-cnn_codigo_seguracampos/data/interim/experiment/test/has_cactus created.
Dif: /home/jascrer/TFM/TFM_DL_CNN_Codigo_SeguraCampos/tfm_dl-cnn_codigo_seguracampos/data/interim/experiment/test/no_cactus created.
Validation export lasted: 21.048335552215576 seconds.
